In [ ]:
import pandas as pd
import polars as pl
import numpy as np
from rtsvg import *
rt = RACETrack()
ofi = rt.ontologyFrameworkInstance().fm_files('../../../data/2024_vast/MC1/2024_rt_ontology')
print(ofi, type(ofi))

In [ ]:
_df_orig_    = ofi.df_triples

# Only keep events
vrbs_to_keep = set()
for _vrb_ in set(_df_orig_['vrb']):
    if _vrb_.startswith('Event.'): vrbs_to_keep.add(_vrb_)
_df_ = _df_orig_.filter(pl.col('vrb').is_in(vrbs_to_keep))
print(f'Filtered ... {len(_df_)=}')

# Add in the actual entity names
_df_ = _df_.with_columns(pl.col('sbj').map_elements(lambda x: ofi.uid_lu[x][0], return_dtype=pl.String).alias('sbj_str'),
                         pl.col('obj').map_elements(lambda x: ofi.uid_lu[x][0], return_dtype=pl.String).alias('obj_str'))
_df_ = _df_.drop(['sdisp', 'odisp', 'grp', 'gdisp', 'src'])

# Add in the edit date
_df_join_ = _df_orig_.filter(pl.col('vrb') == 'Property.dateAdded').with_columns(pl.col('obj').map_elements(lambda x: ofi.uid_lu[x][0], return_dtype=pl.String).alias('timestamp'))
_df_join_ = _df_join_.drop(['uid', 'stype', 'sdisp', 'vrb', 'obj', 'otype', 'odisp', 'grp', 'gdisp', 'src'])
_df_      = _df_.join(_df_join_, left_on='uid', right_on='sbj')
print(f'Edit Date Joined ... {len(_df_)=}')

# Add in the last edited by
_df_join_ = _df_orig_.filter(pl.col('vrb') == 'Property.lastEditedBy').with_columns(pl.col('obj').map_elements(lambda x: ofi.uid_lu[x][0], return_dtype=pl.String).alias('last_edited_by'))
_df_join_ = _df_join_.drop(['uid', 'stype', 'sdisp', 'vrb', 'obj', 'otype', 'odisp', 'grp', 'gdisp', 'src'])
_df_      = _df_.join(_df_join_, left_on='uid', right_on='sbj')
print(f'Last Edited By Joined ... {len(_df_)=}')

# Convet the timestamp string to a timestamp
_df_      = rt.columnsAreTimestamps(_df_, 'timestamp')

rt.temporalBarChart(_df_, color_by='vrb')

In [ ]:
rt.smallMultiples(_df_, category_by='last_edited_by', sm_type='temporalBarChart', color_by='vrb', w=1536, w_sm_override=128, h_sm_override=80)

In [ ]:
rt.smallMultiples(_df_, category_by='obj_str', sm_type='temporalBarChart', color_by='vrb', w=1536, w_sm_override=128, h_sm_override=80)